In [1]:
## Import packages
import os
import csv
import numpy as np
import pandas as pd
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
wavfile_path = 'elephants/angela_elephants'
#wavfile_path = 'elephants/angela_background'

labels_csv = "models/class_labels_indices_jungle.csv"

In [3]:
flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

flags.DEFINE_string(
    'model_checkpoint', 'models/models_jungle/model.ckpt',
    'Path to the VGGish checkpoint file.')

In [4]:
## Read csv-file with labels
class_map = {}
with open(labels_csv) as f:
    next(f)  # skip header
    reader = csv.reader(f)
    for row in reader:
        class_map[int(row[0])] = row[2]

In [5]:
class_map

{0: 'Car, boat, motorcyle, truck',
 1: 'Chainsaw',
 2: 'Chop',
 3: 'Gunshot, gunfire',
 4: 'Helicopter',
 5: 'Male singing',
 6: 'Cattle',
 7: 'Radio',
 8: 'Shout',
 9: 'Silence',
 10: 'Wind noise (microphone)',
 11: 'Gunshot SC9mm',
 12: 'Gunshot SC556',
 13: 'Elephant_Wenger',
 14: 'Bush_Wenger',
 15: 'Gun_Google'}

In [6]:
## Ik verdeel het parsen van een wav-file naar embeddings in stappen:
# Stap 1a: lezen van wav-file, input is array met samples die db aanduiden. Ook sample rate (per sec) wordt gelezen
# Stap 1b: Bij 2d array (stereo, ipv mono) bereken gemiddelde, daarna normaliseren (delen door 32.768)
# Stap 2: Bepaal examples in vorm [batch size, num frames, num bands].
    # Hierbij worden voor verschillende batches (omdat alles tegelijk niet in 1x in NN kan),
    # een log mel spectrogram gemaakt (in vorm [num_frames, num_bands])
# Stap 3: Bepaal features: nu wordt de embedding laag gemaakt (PCA-components, discreet maken etc)
    # Hiervoor worden model-parameters opgehaald die eerder zijn opgeslagen
# Stap 4: Maken van predictions

In [7]:
def getPredictions(examples_batch):    
    ## Stap 3
    with tf.Graph().as_default(), tf.Session() as sess:
        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, vggish_params.VGGISH_MODEL)

        pca_params = np.load(vggish_params.VGGISH_PCA_PARAMS)
        pca_matrix = pca_params[vggish_params.PCA_EIGEN_VECTORS_NAME]
        pca_means = pca_params[vggish_params.PCA_MEANS_NAME].reshape(-1, 1)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)
        vggish_slim.load_youtube_model(sess, FLAGS.model_checkpoint)

        # Run inference and postprocessing
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})

        postprocessed_batch = np.dot(
                pca_matrix, (embedding_batch.T - pca_means)
            ).T
        #print(postprocessed_batch)

        num_frames = np.minimum(postprocessed_batch.shape[0], vggish_params.MAX_FRAMES)
        data = vggish_postprocess.resize(postprocessed_batch, 0, vggish_params.MAX_FRAMES)
        data = np.expand_dims(data, 0)
        num_frames = np.expand_dims(num_frames, 0)

        input_tensor = sess.graph.get_collection("input_batch_raw")[0]
        num_frames_tensor = sess.graph.get_collection("num_frames")[0]
        predictions_tensor = sess.graph.get_collection("predictions")[0]

        ## Stap 4
        predictions_val, = sess.run(
            [predictions_tensor],
            feed_dict={
                input_tensor: data,
                num_frames_tensor: num_frames
            })
    return(predictions_val)

In [8]:
## Filter predictions (give top 20 where p>0.1)
def filterPredictions(predictions_val):
    count = vggish_params.PREDICTIONS_COUNT_LIMIT
    hit = vggish_params.PREDICTIONS_HIT_LIMIT
    top_indices = np.argpartition(predictions_val[0], -count)[-count:]
    line = ((class_map[i], float(predictions_val[0][i])) for i in top_indices if predictions_val[0][i] > hit)
    predictions = sorted(line, key=lambda p: -p[1])
    print(predictions)
    return(predictions)

In [18]:
columns = ["Wav_file"]
for i in range(len(class_map)):
    columns.append(class_map[i])

In [47]:
all_predictions = pd.DataFrame(columns=columns)

In [41]:
# Maak rij aan voor predictions dataframe
def getRow(file, predictions):
    wav_file = file.split('\\')[-1]
    row = [wav_file]
    for prediction in predictions:
        row.append(prediction)
    return(row)

In [48]:
## Read wav files
files = gfile.Glob(str(wavfile_path + "/*.wav"))

i = 0
for file in files:
    ## Stap 1 en 2
    ## This function reads the wav file and converts the samples into np arrays of [batch size, num frames, num bands]
    examples_batch = vggish_input.wavfile_to_examples(file)
    predictions = getPredictions(examples_batch)
    #predictions = filterPredictions(predictions)
    
    row = getRow(file, predictions[0])
    all_predictions.loc[i] = row
    i+=1

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorfl

INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorfl

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorfl

INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorfl

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorfl

...

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorfl

INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorfl

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorfl

INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorfl

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/models_jungle/model.ckpt


In [46]:
all_predictions.head()

,Wav_file,"Car, boat, motorcyle, truck",Chainsaw,Chop,"Gunshot, gunfire",Helicopter,Male singing,Cattle,Radio,Shout,Silence,Wind noise (microphone),Gunshot SC9mm,Gunshot SC556,Elephant_Wenger,Bush_Wenger,Gun_Google
15,ElephantEFAF2011A033-762.WAV,0.006763,0.000077,0.001857,0.013078,0.030711,0.005553,0.000193,0.033585,0.000286,0.398088,0.006513,0.000243,0.000303,0.036462,0.009334,0.010298
0,ElephantEFAF2011A008-112.WAV,0.014229,0.006255,0.005734,0.063009,0.005786,0.010490,0.706854,0.012971,0.002820,0.017671,0.009436,0.000105,0.032853,0.002094,0.001816,0.030892
1,ElephantEFAF2011A008-175.WAV,0.001565,0.004697,0.035026,0.031000,0.001082,0.002457,0.074916,0.010313,0.000510,0.396494,0.004802,0.000111,0.060942,0.005174,0.002084,0.022204
2,ElephantEFAF2011A008-178.WAV,0.002510,0.007060,0.059136,0.042500,0.000985,0.003343,0.132220,0.017600,0.000603,0.339151,0.006199,0.000109,0.066455,0.001552,0.001341,0.030136
3,ElephantEFAF2011A008-180.WAV,0.001438,0.007325,0.063545,0.035672,0.001046,0.002720,0.146794,0.011940,0.000545,0.373182,0.005689,0.000117,0.087913,0.004596,0.001973,0.035890


In [44]:
#from collections import Counter

#sum_occur = 0
#cnt_predictions = Counter(all_predictions)

#for i in cnt_predictions.most_common(10):
#    print(i)

In [52]:
all_predictions.to_csv("csv_files/predictions_angela.csv")